# 导入必要库

In [1]:
from pathlib import Path
import gc
import time

from core import RLEnv
from core.agent import BaseAgent
from greedy import (
    EpsilonDecreasingConfig,
    GreedyAgent,
    greedy_average,
    epsilon_average,
    epsilon_decreasing_average,
)
from ucb1 import UCBAgent, ucb1
from thompson_sampling import TSAgent

from train import batch_train
from utils import plot_metrics_history, save_experiment_data, ProcessDataLogger

In [2]:
STEPS: int = 100_000
GRID_SIZE: int = 500

SEED: int = 42
MACHINE_COUNT: int = 10
RUN_COUNT: int = 500
CONVERGENCE_THRESHOLD: float = 0.9
CONVERGENCE_MIN_STEPS: int = 100
OPTIMISTIC_TIMES: int = 1
ENABLE_OPTIMISTIC: bool = True
EXPERIMENT_DATA_DIR: Path = Path.cwd() / "experiment_data"

ENV: RLEnv = RLEnv(machine_count=MACHINE_COUNT, seed=SEED)
EPSILON_CONFIG: EpsilonDecreasingConfig = EpsilonDecreasingConfig()

# 工厂函数

In [3]:
def get_run_id(agent_name: str) -> str:
    return agent_name + "_" + str(time.time())

In [4]:
def create_greedy_agent(
    env: RLEnv,
    epsilon_config: EpsilonDecreasingConfig,
    optimistic_init: bool,
    optimistic_times: int,
    convergence_threshold: float,
    convergence_min_steps: int,
    seed: int,
) -> BaseAgent:
    return GreedyAgent(
        name=greedy_average.__name__,
        env=env,
        greedy_algorithm=greedy_average,
        epsilon_config=epsilon_config,
        optimistic_init=optimistic_init,
        optimistic_times=optimistic_times,
        convergence_threshold=convergence_threshold,
        convergence_min_steps=convergence_min_steps,
        seed=seed,
    )


def create_epsilon_agent(
    env: RLEnv,
    epsilon_config: EpsilonDecreasingConfig,
    optimistic_init: bool,
    optimistic_times: int,
    convergence_threshold: float,
    convergence_min_steps: int,
    seed: int,
) -> BaseAgent:
    return GreedyAgent(
        name=epsilon_average.__name__,
        env=env,
        greedy_algorithm=epsilon_average,
        epsilon_config=epsilon_config,
        optimistic_init=optimistic_init,
        optimistic_times=optimistic_times,
        convergence_threshold=convergence_threshold,
        convergence_min_steps=convergence_min_steps,
        seed=seed,
    )


def create_decreasing_agent(
    env: RLEnv,
    epsilon_config: EpsilonDecreasingConfig,
    optimistic_init: bool,
    optimistic_times: int,
    convergence_threshold: float,
    convergence_min_steps: int,
    seed: int,
) -> BaseAgent:
    return GreedyAgent(
        name=epsilon_decreasing_average.__name__,
        env=env,
        greedy_algorithm=epsilon_decreasing_average,
        epsilon_config=epsilon_config,
        optimistic_init=optimistic_init,
        optimistic_times=optimistic_times,
        convergence_threshold=convergence_threshold,
        convergence_min_steps=convergence_min_steps,
        seed=seed,
    )

In [5]:
def create_ucb1_agent(
    env: RLEnv,
    convergence_threshold: float,
    convergence_min_steps: int,
    seed: int,
) -> BaseAgent:
    return UCBAgent(
        name=ucb1.__name__,
        env=env,
        ucb1_algorithm=ucb1,
        convergence_threshold=convergence_threshold,
        convergence_min_steps=convergence_min_steps,
        seed=seed,
    )

In [6]:
def create_ts_agent(
    env: RLEnv,
    convergence_threshold: float,
    convergence_min_steps: int,
    seed: int,
) -> BaseAgent:
    return TSAgent(
        name=TSAgent.__name__,
        env=env,
        convergence_threshold=convergence_threshold,
        convergence_min_steps=convergence_min_steps,
        seed=seed,
    )

# 训练

## 普通贪婪算法

In [7]:
run_id = get_run_id(greedy_average.__name__)
file_name: Path = (
    EXPERIMENT_DATA_DIR
    / f"{run_id}_T={STEPS}_K={MACHINE_COUNT}_Q_0={OPTIMISTIC_TIMES}.png"
)
process_logger = ProcessDataLogger(
    run_id=run_id,
    total_steps=STEPS,
    grid_size=GRID_SIZE,
)

agents, reward, metrics = batch_train(
    count=RUN_COUNT,
    agent_factory=create_greedy_agent,
    env=ENV,
    epsilon_config=EPSILON_CONFIG,
    steps=STEPS,
    seed=SEED,
    optimistic_init=ENABLE_OPTIMISTIC,
    optimistic_times=OPTIMISTIC_TIMES,
    convergence_threshold=CONVERGENCE_THRESHOLD,
    convergence_min_steps=CONVERGENCE_MIN_STEPS,
    process_logger=process_logger,
)
print(metrics)
print(reward)

plot_metrics_history(agents, "贪婪算法", file_name, x_log=False)
plot_metrics_history(agents, "贪婪算法", file_name, x_log=True)
save_experiment_data(reward, metrics, file_name)
process_logger.save(file_name.with_stem(file_name.stem + "process"), total_steps=STEPS)
dump = process_logger.export(total_steps=STEPS)
keys = list(dump.points[0].data.keys())

del agents, reward, metrics, process_logger, dump
gc.collect()

达到收敛时的步数: 190
达到收敛时的步数: 510
达到收敛时的步数: 250
达到收敛时的步数: 130
达到收敛时的步数: 560
达到收敛时的步数: 150
达到收敛时的步数: 160
达到收敛时的步数: 170
达到收敛时的步数: 150
达到收敛时的步数: 200
达到收敛时的步数: 150
达到收敛时的步数: 220
达到收敛时的步数: 170
达到收敛时的步数: 160
达到收敛时的步数: 150
达到收敛时的步数: 200
达到收敛时的步数: 500
达到收敛时的步数: 150
达到收敛时的步数: 160
达到收敛时的步数: 130
达到收敛时的步数: 290
达到收敛时的步数: 470
达到收敛时的步数: 410
达到收敛时的步数: 250
达到收敛时的步数: 230
达到收敛时的步数: 540
达到收敛时的步数: 210
达到收敛时的步数: 170
达到收敛时的步数: 350
达到收敛时的步数: 110
达到收敛时的步数: 170
达到收敛时的步数: 220
达到收敛时的步数: 1100
达到收敛时的步数: 270
达到收敛时的步数: 150
达到收敛时的步数: 170
达到收敛时的步数: 290
达到收敛时的步数: 500
达到收敛时的步数: 190
达到收敛时的步数: 120
达到收敛时的步数: 150
达到收敛时的步数: 200
达到收敛时的步数: 140
达到收敛时的步数: 180
达到收敛时的步数: 230
达到收敛时的步数: 210
达到收敛时的步数: 320
达到收敛时的步数: 280
达到收敛时的步数: 160
达到收敛时的步数: 330
达到收敛时的步数: 120
达到收敛时的步数: 120
达到收敛时的步数: 230
达到收敛时的步数: 840
达到收敛时的步数: 640
达到收敛时的步数: 220
达到收敛时的步数: 210
达到收敛时的步数: 350
达到收敛时的步数: 190
达到收敛时的步数: 810
达到收敛时的步数: 270
达到收敛时的步数: 280
达到收敛时的步数: 220
达到收敛时的步数: 300
达到收敛时的步数: 140
达到收敛时的步数: 410
达到收敛时的步数: 230
达到收敛时的步数: 210
达到收敛时的步数: 170
达到收敛时的步数: 510
达到收敛时的步数: 160
达到收敛时

33

## UCB1算法

In [8]:
run_id = get_run_id(ucb1.__name__)
file_name: Path = (
    EXPERIMENT_DATA_DIR
    / f"{run_id}_T={STEPS}_K={MACHINE_COUNT}_Q_0={OPTIMISTIC_TIMES}.png"
)
process_logger = ProcessDataLogger(
    run_id=run_id,
    total_steps=STEPS,
    grid_size=GRID_SIZE,
)

agents, reward, metrics = batch_train(
    count=RUN_COUNT,
    agent_factory=create_ucb1_agent,
    env=ENV,
    steps=STEPS,
    seed=SEED,
    convergence_threshold=CONVERGENCE_THRESHOLD,
    convergence_min_steps=CONVERGENCE_MIN_STEPS,
    process_logger=process_logger,
)
print(metrics)
print(reward)

plot_metrics_history(agents, "UCB1 算法", file_name, x_log=False)
plot_metrics_history(agents, "UCB1 算法", file_name, x_log=True)
save_experiment_data(reward, metrics, file_name)
process_logger.save(file_name.with_stem(file_name.stem + "process"), total_steps=STEPS)
dump = process_logger.export(total_steps=STEPS)
keys = list(dump.points[0].data.keys())

del agents, reward, metrics, process_logger, dump
gc.collect()

达到收敛时的步数: 19160
达到收敛时的步数: 26890
达到收敛时的步数: 29670
达到收敛时的步数: 31410
达到收敛时的步数: 22520
达到收敛时的步数: 23260
达到收敛时的步数: 24000
达到收敛时的步数: 24860
达到收敛时的步数: 33050
达到收敛时的步数: 20300
达到收敛时的步数: 28920
达到收敛时的步数: 26570
达到收敛时的步数: 21430
达到收敛时的步数: 15080
达到收敛时的步数: 27750
达到收敛时的步数: 32190
达到收敛时的步数: 26390
达到收敛时的步数: 33970
达到收敛时的步数: 27130
达到收敛时的步数: 30480
达到收敛时的步数: 25760
达到收敛时的步数: 24710
达到收敛时的步数: 29450
达到收敛时的步数: 26360
达到收敛时的步数: 29750
达到收敛时的步数: 17070
达到收敛时的步数: 27790
达到收敛时的步数: 27440
达到收敛时的步数: 22700
达到收敛时的步数: 21200
达到收敛时的步数: 25550
达到收敛时的步数: 23230
达到收敛时的步数: 26890
达到收敛时的步数: 20870
达到收敛时的步数: 33370
达到收敛时的步数: 31720
达到收敛时的步数: 21330
达到收敛时的步数: 20890
达到收敛时的步数: 29790
达到收敛时的步数: 24010
达到收敛时的步数: 20310
达到收敛时的步数: 23550
达到收敛时的步数: 22140
达到收敛时的步数: 27790
达到收敛时的步数: 27630
达到收敛时的步数: 29540
达到收敛时的步数: 17270
达到收敛时的步数: 29590
达到收敛时的步数: 25710
达到收敛时的步数: 28470
达到收敛时的步数: 28880
达到收敛时的步数: 29100
达到收敛时的步数: 20840
达到收敛时的步数: 22160
达到收敛时的步数: 26960
达到收敛时的步数: 25040
达到收敛时的步数: 25050
达到收敛时的步数: 22450
达到收敛时的步数: 22980
达到收敛时的步数: 26880
达到收敛时的步数: 25270
达到收敛时的步数: 18860
达到收敛时的步数

33

# Thompson Sampling 算法

In [9]:
run_id = get_run_id("thompson_sampling")
file_name: Path = (
    EXPERIMENT_DATA_DIR
    / f"{run_id}_T={STEPS}_K={MACHINE_COUNT}_Q_0={OPTIMISTIC_TIMES}.png"
)
process_logger = ProcessDataLogger(
    run_id=run_id,
    total_steps=STEPS,
    grid_size=GRID_SIZE,
)

agents, reward, metrics = batch_train(
    count=RUN_COUNT,
    agent_factory=create_ts_agent,
    env=ENV,
    steps=STEPS,
    seed=SEED,
    convergence_threshold=CONVERGENCE_THRESHOLD,
    convergence_min_steps=CONVERGENCE_MIN_STEPS,
    process_logger=process_logger,
)
print(metrics)
print(reward)

plot_metrics_history(agents, "TS 算法", file_name, x_log=False)
plot_metrics_history(agents, "TS 算法", file_name, x_log=True)
save_experiment_data(reward, metrics, file_name)
process_logger.save(file_name.with_stem(file_name.stem + "process"), total_steps=STEPS)
dump = process_logger.export(total_steps=STEPS)
keys = list(dump.points[0].data.keys())

del agents, reward, metrics, process_logger, dump
gc.collect()

达到收敛时的步数: 530
达到收敛时的步数: 1580
达到收敛时的步数: 770
达到收敛时的步数: 600
达到收敛时的步数: 1170
达到收敛时的步数: 1370
达到收敛时的步数: 700
达到收敛时的步数: 1430
达到收敛时的步数: 4150
达到收敛时的步数: 2360
达到收敛时的步数: 1590
达到收敛时的步数: 1480
达到收敛时的步数: 900
达到收敛时的步数: 880
达到收敛时的步数: 590
达到收敛时的步数: 340
达到收敛时的步数: 1970
达到收敛时的步数: 2210
达到收敛时的步数: 340
达到收敛时的步数: 1990
达到收敛时的步数: 1230
达到收敛时的步数: 940
达到收敛时的步数: 3300
达到收敛时的步数: 710
达到收敛时的步数: 1080
达到收敛时的步数: 840
达到收敛时的步数: 700
达到收敛时的步数: 2370
达到收敛时的步数: 570
达到收敛时的步数: 470
达到收敛时的步数: 1480
达到收敛时的步数: 1590
达到收敛时的步数: 1960
达到收敛时的步数: 2600
达到收敛时的步数: 770
达到收敛时的步数: 1540
达到收敛时的步数: 930
达到收敛时的步数: 810
达到收敛时的步数: 1400
达到收敛时的步数: 700
达到收敛时的步数: 1040
达到收敛时的步数: 740
达到收敛时的步数: 1910
达到收敛时的步数: 880
达到收敛时的步数: 1290
达到收敛时的步数: 1520
达到收敛时的步数: 1100
达到收敛时的步数: 500
达到收敛时的步数: 410
达到收敛时的步数: 680
达到收敛时的步数: 820
达到收敛时的步数: 660
达到收敛时的步数: 1250
达到收敛时的步数: 1880
达到收敛时的步数: 450
达到收敛时的步数: 1610
达到收敛时的步数: 1460
达到收敛时的步数: 2070
达到收敛时的步数: 1660
达到收敛时的步数: 3530
达到收敛时的步数: 1420
达到收敛时的步数: 630
达到收敛时的步数: 1660
达到收敛时的步数: 1080
达到收敛时的步数: 2030
达到收敛时的步数: 930
达到收敛时的步数: 1300
达到收敛时的步数: 1670
达到收敛时的步数:

2280814